<a href="https://colab.research.google.com/github/gugupig/Seq2seq/blob/main/Seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Downloading the dataset



In [ ]:
# Imports Pytorch.
import torch
import torch.nn as nn
from nltk.translate.bleu_score import sentence_bleu

In [ ]:
#Downloads the dataset.
#use 2 books to train
import urllib
books  =  [urllib.request.urlretrieve("https://www.gutenberg.org/cache/epub/14155/pg14155.txt")[0],urllib.request.urlretrieve("https://www.gutenberg.org/cache/epub/17489/pg17489.txt")[0]]
for d in books:
  print(d)

/tmp/tmpq6j7vrcn
/tmp/tmpdnkzs05v


In [ ]:
# Prints the first 200 lines in the file with their line number.
# This shows that we have a little bit of preprocessing to do in order to clean the data.
filename  = books[0]
with open(filename,encoding="utf-8") as f:
  for i in range(200):
    print(f"[{i}] {f.readline()}", end='')

[0] ﻿The Project Gutenberg EBook of Madame Bovary, by Gustave Flaubert
[1] 
[2] This eBook is for the use of anyone anywhere at no cost and with
[3] almost no restrictions whatsoever.  You may copy it, give it away or
[4] re-use it under the terms of the Project Gutenberg License included
[5] with this eBook or online at www.gutenberg.org
[6] 
[7] 
[8] Title: Madame Bovary
[9] 
[10] Author: Gustave Flaubert
[11] 
[12] Release Date: November 26, 2004 [EBook #14155]
[13] [Last updated: November 28, 2011]
[14] 
[15] 
[16] Language: French
[17] 
[18] 
[19] *** START OF THIS PROJECT GUTENBERG EBOOK MADAME BOVARY ***
[20] 
[21] 
[22] 
[23] 
[24] Produced by Ebooks libres et gratuits at http://www.ebooksgratuits.com
[25] 
[26] 
[27] 
[28] 
[29] 
[30] Gustave Flaubert
[31] MADAME BOVARY
[32] 
[33] 
[34] (1857)
[35] 
[36] 
[37] Table des matières
[38] 
[39] PREMIÈRE PARTIE
[40] I
[41] II
[42] III
[43] IV
[44] V
[45] VI
[46] VII
[47] VIII
[48] IX
[49] DEUXIÈME PARTIE
[50] I
[51] II
[52] III
[53]

In [ ]:
skiping_para = ["PREMIÈRE PARTIE","Livre premier--Un juste"]
end_para = ["End of the Project Gutenberg EBook of Madame Bovary, by Gustave Flaubert","End of the Project Gutenberg EBook of Les misérables Tome I, by Victor Hugo"]
import re # Regular expression library
roman_regex = re.compile('^M{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})$') # This regular expression matches Roman numerals but also the empty string.
EOP = '\n' # The end-of-line character will be used to mark the end of paragraphs.
book_1  =  books[0]
book_2  = books[1]

def make_paragraphs(filename):
  
  with open(filename,encoding="utf-8") as f:
    # skip everything before the actual text of the novel.
    # The line "PREMIÈRE PARTIE" appears twice: in the table of content and then at the start of the first part of the actual text.
    # The following lines discard everything up to this second occurence (included).
    skip = 2
    while(skip > 0):
      line = f.readline().strip()
      if(line in skiping_para): skip -= 1;

    paragraphs = [] # each dialog line will be considered a separate paragraph.
    paragraph_buffer = [] # List[str]; each element corresponds to a line in the original text file + an additonal space if necessary.
    while(True):
      line = f.readline().strip()
      if(line in end_para): break # End of the actual text.

      if(line == ""): #the end of a paragraph.
        if(len(paragraph_buffer) > 0):
          paragraph_buffer.append(EOP) # End of the paragraph.

          paragraph = "".join(paragraph_buffer) # The different lines that make up the paragraph are joined into a single string.
          paragraphs.append(paragraph)
          paragraph_buffer = []
        continue

      if(roman_regex.match(line)): continue # Ignores the lines that indicate the beginning of a chapter.
      if(line.endswith(" PARTIE")): continue # Ignores the lines that indicate the beginning of a part.

      if((len(paragraph_buffer) > 0) and (paragraph_buffer[-1][-1] != '-')): paragraph_buffer.append(' ') # Adds a space between consecutive lines except when the first one ends with "-" (e.g. if the word "pomme-de-terre" is split with "pomme-de-" at the end of a line and "terre" at the beginning of the next, we do not want to join the two lines with a space).
      paragraph_buffer.append(line)
    return paragraphs


paragraphs  = make_paragraphs(book_1)
paragraphs_2  = make_paragraphs(book_2)
print(f"{len(paragraphs)} paragraphs read.")
for i in range(3): print(paragraphs[i], end='')

2995 paragraphs read.
Nous étions à l'Étude, quand le Proviseur entra, suivi d'un nouveau habillé en bourgeois et d'un garçon de classe qui portait un grand pupitre. Ceux qui dormaient se réveillèrent, et chacun se leva comme surpris dans son travail.
Le Proviseur nous fit signe de nous rasseoir; puis, se tournant vers le maître d'études:
-- Monsieur Roger, lui dit-il à demi-voix, voici un élève que je vous recommande, il entre en cinquième. Si son travail et sa conduite sont méritoires, il passera dans les grands, où l'appelle son âge.


In [ ]:
import collections
import numpy as np

# Computes the frequency of all characters in the dataset.
def  count_char(g):
    char_counts = collections.defaultdict(int)
    for paragraph in g:
      for char in paragraph: char_counts[char] += 1
    return char_counts

paragraphs_all = paragraphs.copy()
paragraphs_all.extend(paragraphs_2)
char_counts = count_char (paragraphs_all)
print(f"{len(char_counts)} different characters found in the dataset.")
print(sorted(char_counts.items(), key=(lambda x: x[1]), reverse=True)) # Shows each character with its frequency, in decreasing frequency order.


102 different characters found in the dataset.
[(' ', 216077), ('e', 151088), ('a', 86232), ('s', 80014), ('t', 77066), ('i', 75967), ('n', 69471), ('r', 65834), ('u', 64636), ('l', 62639), ('o', 52942), ('d', 36323), ('c', 28988), ('m', 28812), ('p', 26509), (',', 21198), ('é', 18018), ('v', 17966), ("'", 15056), ('.', 12908), ('q', 11734), ('f', 10856), ('h', 10329), ('b', 10209), ('g', 9056), ('-', 8446), ('\n', 5873), ('à', 5392), ('x', 4063), ('j', 3775), ('y', 3326), ('è', 3324), ('ê', 2758), ('!', 2375), (';', 2267), ('C', 2156), ('L', 2145), ('I', 2108), ('E', 2096), ('M', 1925), ('z', 1615), ('?', 1195), (':', 1097), ('J', 1057), ('A', 1007), ('P', 894), ('D', 846), ('â', 816), ('ç', 813), ('V', 751), ('S', 742), ('B', 680), ('O', 637), ('T', 620), ('î', 602), ('ù', 576), ('û', 576), ('ô', 567), ('Q', 559), ('_', 556), ('F', 465), ('R', 401), ('N', 340), ('«', 335), ('H', 330), ('U', 323), ('»', 262), ('À', 203), ('G', 185), ('1', 124), ('Y', 89), ('É', 84), ('ï', 83), ('(', 6

In [ ]:
# build a dictionary 'char_vocabulary' that assigns an integer id to each character, along with list/array 'id_to_char' that implement the reverse mapping.
#################
def make_maps(char_counts):
    id_to_char  = []
    char_vocabulary =  {}
    for char in char_counts.keys():
      id_to_char.append(char)
    id_to_char.remove(EOP) # [Timothée Bernard] ?? à expliquer
    id_to_char.append(EOP) # [Timothée Bernard] ?? à expliquer
    id_to_char.append('<p>') # [Timothée Bernard] ?? à expliquer
    for char in id_to_char:
      char_vocabulary[char] = id_to_char.index(char)
    return id_to_char,char_vocabulary
id_to_char,char_vocabulary = make_maps(char_counts)
#################

In [ ]:
EOP_id = char_vocabulary[EOP] # Id for the end-of-paragraph symbol
PAD_id = char_vocabulary['<p>']

print(f"EOP_id = {EOP_id}")

EOP_id = 101


In [ ]:
# Turns a list of lists of ids into a list of strings.
def ids_to_texts(ids):
  # Turn each list of character ids of 'ids' into a string and then return all strings as a list.
  #################
  return ["".join([''.join([id_to_char[id]]) for id in l]) for l in ids]
  #################

In [ ]:
ps = ["Bonjour.", "Comment allez vous ?"]
ids = [[char_vocabulary[c] for c in p] for p in ps]
print(ids)
print(ids_to_texts(ids))
print(f"'ids_to_texts(ids) == ps' should be True: {ids_to_texts(ids) == ps}")

[[60, 1, 8, 40, 1, 2, 19, 27], [28, 1, 30, 30, 14, 8, 6, 4, 17, 10, 10, 14, 44, 4, 20, 1, 2, 3, 4, 61]]
['Bonjour.', 'Comment allez vous ?']
'ids_to_texts(ids) == ps' should be True: True


In [ ]:
ps = ["Bonjour.", "Comment allez vous ?"]
ids = [[char_vocabulary[c] for c in p] for p in ps]
ids[0].extend([EOP_id, (EOP_id+1), (EOP_id+1)]) # With the end-of-paragraph token id and some padding ids for the first string.
print(ids)
print(ids_to_texts(ids))
print(f"'ids_to_texts(ids) == ps' should be True: {ids_to_texts(ids) == ps}") # [Timothée Bernard] Le test n'est pas réussi mais vous vous en moquez ?

[[60, 1, 8, 40, 1, 2, 19, 27, 101, 102, 102], [28, 1, 30, 30, 14, 8, 6, 4, 17, 10, 10, 14, 44, 4, 20, 1, 2, 3, 4, 61]]
['Bonjour.\n<p><p>', 'Comment allez vous ?']
'ids_to_texts(ids) == ps' should be True: False


Batch generator
==

In [ ]:
# Defines a class of objects that produce batches from the dataset.
# A training instance is composed of a pair of consecutive paragraphs. The goal will be to predict the second given the first.
# (Possible improvement: As is, ends of chapter are completely ignored: the last paragraph of a chapter and the first of the following chapter form a training instance. We might want to predict the end of the chapter instead, or simply remove these pairs from the dataset.)
class BatchGenerator:
  def __init__(self, paragraphs, char_vocabulary):
    self.paragraphs = paragraphs
    self.char_vocabulary = char_vocabulary # Dictionary
    self.padding_idx = len(char_vocabulary) - 1
  
  # Returns the number of training instances (i.e. of pairs of consecutive paragraphs).
  def length(self):
    return (len(self.paragraphs) - 1)

  # Returns a random training batch (composed of pairs of consecutive paragraphs).
  # If `subset` is an integer, only a subset of the corpus is used. This can be useful to debug the system.
  def get_batch(self, batch_size, subset=None):
    max_i = self.length() if(subset is None) else min(subset, self.length())
    paragraph_ids = np.random.randint(max_i, size=batch_size) # Randomly picks some paragraph ids.

    return self._ids_to_batch(paragraph_ids)

  def _ids_to_batch(self, paragraph_ids):
    firsts = [] # First paragraph of each pair
    seconds = [] # Second paragraph of each pair
    for paragraph_id in paragraph_ids:
      firsts.append([self.char_vocabulary[char] for char in self.paragraphs[paragraph_id]])
      seconds.append([self.char_vocabulary[char] for char in self.paragraphs[paragraph_id + 1]])
      
    # Padding
    self.pad(firsts)
    self.pad(seconds)

    firsts = torch.tensor(firsts, dtype=torch.long) # Conversion to a tensor
    seconds = torch.tensor(seconds, dtype=torch.long) # Conversion to a tensor

    return (firsts, seconds)
  
  # Pads a list of lists (i.e. adds fake word ids so that all sequences in the batch have the same length, so that we can use a matrix to represent them).
  # In place
  def pad(self, sequences):
    max_length = max([len(s) for s in sequences])
    for s in sequences: s.extend([self.padding_idx] * (max_length - len(s)))
  
  # Returns a generator of training batches for a full epoch.
  # If `subset` is an integer, only a subset of the corpus is used. This can be useful to debug the system.
  def all_batches(self, batch_size, subset=None):
    max_i = self.length() if(subset is None) else min(subset, self.length())

    # Loop that generates all full batches (batches of size 'batch_size')
    i = 0
    while((i + batch_size) <= max_i):
      instance_ids = np.arange(i, (i + batch_size))
      yield self._ids_to_batch(instance_ids)
      i += batch_size
    
    # Possibly generates the last (not full) batch.
    if(i < max_i):
      instance_ids = np.arange(i, max_i)
      yield self._ids_to_batch(instance_ids)
  
  # Turns a list of arbitrary paragraphs into a prediction batch.
  def turn_into_batch(self, paragraphs):
    firsts = []
    for paragraph in paragraphs:
        # Unknown characters are ignored (removed).
        tmp = []
        for char in paragraph:
          if(char in self.char_vocabulary): tmp.append(self.char_vocabulary[char])

        if(tmp[-1] != EOP_id): tmp.append(EOP_id) # Adds an end-of-paragraph character if necessary.

        firsts.append(tmp)
    
    self.pad(firsts)
    return torch.tensor(firsts, dtype=torch.long)

batch_generator = BatchGenerator(paragraphs=paragraphs, char_vocabulary=char_vocabulary)
print(batch_generator.length())

2994


In [ ]:
(firsts, seconds) = batch_generator.get_batch(3)
print(firsts.size())
print(seconds.size())
print(ids_to_texts(firsts))
print(ids_to_texts(seconds))

torch.Size([3, 252])
torch.Size([3, 214])
['-- Mais enfin, monsieur, fit Emma, vous aviez à me dire...?\n<p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p>', '-- Avec un sac de nuit.\n<p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p><p

The model
==
(Don't forget to read carefully all comments and to make sure you understand them.)

In [ ]:
class Model(torch.nn.Module):
  # 'size_vocabulary' does not include a padding character, but does include the end-of-paragraph one.
  def __init__(self, vocabulary_size, EOP_id, embedding_size, lstm_hidden_size, lstm_layers, encoder_dropout_p = 0,decoder_dropout_p = 0 ,device='cpu'):
    super().__init__()

    self.vocabulary_size = vocabulary_size
    self.EOP_id = EOP_id
    self.embedding_size = embedding_size
    self.hidden_size = lstm_hidden_size
    self.lstm_layers = lstm_layers
    self.encoder_dropout_p = encoder_dropout_p
    self.decoder_dropout_p =  decoder_dropout_p
    self.device = device



    # At prediction time, this index is used to stop the generation at the end of the paragraph.
    self.char_embeddings = nn.Embedding(self.vocabulary_size,
                                        self.embedding_size,
                                        padding_idx = self.vocabulary_size -1
                                        ).to(self.device)
    self.encoder_lstm = nn.LSTM(input_size =  self.embedding_size,
                                hidden_size = self.hidden_size,
                                batch_first = True,
                                bidirectional = True,
                                num_layers = self.lstm_layers,
                                dropout = self.encoder_dropout_p).to(self.device)
    self.decoder_lstm = nn.LSTM(input_size= self.embedding_size,
                                hidden_size = self.hidden_size,
                                batch_first=True,                             
                                num_layers = self.lstm_layers,
                                dropout  = self.decoder_dropout_p).to(self.device)               
    self.decoder_initialiser = nn.Linear(in_features= self.hidden_size*2,out_features= self.hidden_size).to(self.device)
    self.distribution_nn = nn.Sequential(*[nn.Linear(in_features= self.hidden_size,out_features= self.vocabulary_size),
                                        nn.LogSoftmax(dim=-1)
                                        ]).to(self.device)
    self.dropout = nn.Dropout(self.encoder_dropout_p).to(self.device)




    #################

  # 'in_paragraphs' is a matrix (batch size, max in length) of character ids (Integer).
  # 'out_paragraphs' is None at prediction time or a matrix (batch size, max out length) of character ids (Integer) at training time. Assume it does not include the final end-of-paragraph character.
  
  #this is for beam search,it build top k candidate at each time step (k is the beam width)
  #it returns a list of k tuples : (candidate's real vocab_id,it's cumulative log probability,the beam position of previous candidate who generate this candidate)
  def find_topk (self,cumulative_logp,beam_size):
        flatten = cumulative_logp.view(1,-1)
        topk_logp,topkid = torch.topk(flatten,k=beam_size)
        topk_logp = topk_logp.squeeze() if beam_size >1 else topk_logp
        topkid = topkid.squeeze() if beam_size >1 else topkid
        new_candidate = []
        if beam_size > 1:
            for i in range(0,beam_size):
              beampos = int((topkid[i])/cumulative_logp.size(1))
              real_id = topkid[i]-beampos*cumulative_logp.size(1)
              new_candidate.append((real_id,topk_logp[i],beampos))
        else:
            new_candidate.append((topkid,topk_logp,0))
        return new_candidate

      #Build a text_id list given the initial_candidate
      #Since beampos is an indication of the parent node,it uses this information to retrieve text_ids from the initial_candidate
  def build_from_candidate(self,candidates,initial_candidate,initial_index):
        text_ids = [initial_candidate[0]] if initial_candidate[0] != self.EOP_id else []
        beampos = initial_candidate[2]
        candidates_slice  = candidates[:initial_index][::-1]
        for c in candidates_slice:
          if c[beampos][0] == self.EOP_id:
              break
          text_ids.append(c[beampos][0])
          beampos = c[beampos][2]
        return text_ids[::-1]

      #Get all possible predictions from candidates
      #it return a list of tuples (log_p,text_ids)
  def get_all_predictions(self,candidates,EOP_id):
        predictions = []
        index = 0
        #Add the best prediction from the last time step:
        best_guess = max(candidates[-1],key = lambda x : x[1])
        predictions.append((best_guess[1],self.build_from_candidate(candidates,best_guess,len(candidates)-1)))

        #Add all other possible predictions with EOP mark
        for c in candidates:
            index +=  1
            for tup in c:
                if tup[0] == EOP_id:
                    predictions.append((tup[1],self.build_from_candidate(candidates,tup,index)))

        return predictions
    
  def add_first_char(self,out_paragraphs): # [Timothée Bernard] Que fait cette fonction ?
        batch_size,max_length = out_paragraphs.size(0),out_paragraphs.size(1)
        flatten = out_paragraphs.reshape(1,batch_size * max_length)[0] # [Timothée Bernard] ???????????????
        first = torch.tensor([self.EOP_id],dtype=torch.long).to(self.device)
        flatten = torch.cat((first,flatten[:-1])).to(self.device)
        flatten = flatten.reshape(batch_size,max_length)
        return flatten.to(self.device)
  
  def find_last_char(self,l):
    i = len(l)-1
    while l[i]==self.vocabulary_size or l[i]==0:
      i-=1
    return l[i]  


  def forward(self, in_paragraphs, out_paragraphs=None, max_predicted_char=1000,beam_size = 1):
    batch_size = in_paragraphs.size(0)

    in_char_embeddings = self.dropout(self.char_embeddings(in_paragraphs)) # Shape: (batch_size, max length, embedding size)
    #print(in_char_embeddings); print(in_char_embeddings.shape)
    in_lengths = (in_paragraphs != self.char_embeddings.padding_idx).sum(axis=1) # Shape: (batch_size)
    #print(in_lengths); print(in_lengths.shape)
    in_char_embeddings = torch.nn.utils.rnn.pack_padded_sequence(input=in_char_embeddings, lengths=in_lengths.cpu(), batch_first=True, enforce_sorted=False) # Enables the LSTM to ignore padding elements.

    # The input paragraphs are encoded; the final hidden and cell states of the network will be used to initialise the decoder after a little transformation.
    _, (h_n, c_n) = self.encoder_lstm(in_char_embeddings) # 'h_n' and 'c_n' are both of shape (num_layers * 2, batch_size, lstm_unit)

    # Concatenates the left-to-right and right-to-left final hidden states of the biLSTM.
    h_n = h_n.view(self.encoder_lstm.num_layers, 2, batch_size, self.encoder_lstm.hidden_size) # The second dimension corresponds to left-to-right (0) and right-to-left (1).
    #print(h_n); print(h_n.shape)
    lr_h_n = h_n[:,0] # left-to-right; shape: (num_layers, batch_size, lstm_unit)
    rl_h_n = h_n[:,1] # right-to-left; shape: (num_layers, batch_size, lstm_unit)
    bi_h_n = torch.cat([lr_h_n, rl_h_n], axis=2) # Shape: (num_layers, batch_size, 2 * lstm_unit)
    #print(bi_h_n); print(bi_h_n.shape)

    # Concatenates the left-to-right and right-to-left final cell states of the biLSTM.
    c_n = c_n.view(self.encoder_lstm.num_layers, 2, batch_size, self.encoder_lstm.hidden_size) # The second dimension corresponds to left-to-right (0) and right-to-left (1).
    #print(c_n); print(c_n.shape)
    lr_c_n = c_n[:,0] # left-to-right; shape: (num_layers, batch_size, lstm_unit)
    rl_c_n = c_n[:,1] # right-to-left; shape: (num_layers, batch_size, lstm_unit)
    bi_c_n = torch.cat([lr_c_n, rl_c_n], axis=2) # Shape: (num_layers, batch_size, 2 * lstm_unit)
    #print(bi_c_n); print(bi_c_n.shape)

    last_char = self.find_last_char(in_paragraphs[-1])
    decoder_init_states = (self.decoder_initialiser(bi_h_n), self.decoder_initialiser(bi_c_n)) # These tensors are not only used to initialise the decoder but also (for the first tensor) to compute the probability distributions for the first character.
    if(out_paragraphs is not None): # Training time: outputs the logits for each time step.
      # Feed a packed sequence to the decoder (use 'torch.nn.utils.rnn.pack_padded_sequence' and 'torch.nn.utils.rnn.pad_packed_sequence').
      #################
      out_paragraphs = self.add_first_char(out_paragraphs)
      out_char_embeddings = self.dropout((self.char_embeddings(out_paragraphs)))
      out_lengths = (out_paragraphs != self.char_embeddings.padding_idx).sum(axis=1) # Shape: (batch_size)
      out_char_embeddings = nn.utils.rnn.pack_padded_sequence(input=out_char_embeddings, lengths=out_lengths.cpu(), batch_first=True, enforce_sorted=False)
      out,_ = self.decoder_lstm(out_char_embeddings,decoder_init_states)
      out =  nn.utils.rnn.pad_packed_sequence(out, batch_first=True)[0]
      prediction = self.distribution_nn(out)
      #################

    else: # Prediction time: generates a text up to 'max_predicted_char' character long for each paragraph in the batch.
      # Decode 'decoder_init_states' into a matrix a character ids (on line per input paragraph in the batch) and then convert it to string of actual characters.
      #################
      #Vectorized beam search : batch_size = beam_size,set beam_size to 1 to switch to the greedy search
      
      dc_initial_token = self.char_embeddings(last_char).view(1,1,-1)
      if decoder_init_states[0].size(1) > 1:
        raise Exception ('Text generation do not support batch,please keep batch size = 1')
      first_out,(h_n_dc,c_n_dc) = self.decoder_lstm(dc_initial_token,decoder_init_states) #h,c : nb_layers,beam_size,hidden_size
      #expand hidden and cell to beam size
      h_n_dc = h_n_dc.expand(-1,beam_size,-1).contiguous() if beam_size > 1 else h_n_dc
      c_n_dc = c_n_dc.expand(-1,beam_size,-1).contiguous() if beam_size > 1 else c_n_dc
      first_out = self.distribution_nn(first_out) #first_out  : batch_size = 1,seq_len = 1,vocab_size
      topk_logp,topk_id = torch.topk(first_out,k = beam_size)
      #candidates is a list of topk log probability item in each time step,each candidate is a tuple of (candidate's vocab_id,candidate'scumulative log probability,candidate's "beam position")
      #beampos(beam position) is a interger that points to the posistion of previous candidate (a.k.a parent node) who generates the current candidate,so it always < beam width
      #beampos is crucial for the navigation
      #beam_batch is a list of topk ids at each time step,it used as the input for the next time step (beam_batch is not necessarily contains all last candidates)
      if beam_size == 1:
        candidates = [[(topk_id.squeeze(),topk_logp.squeeze(),0)]]
        beam_batch = [topk_id.squeeze()]
      else:
        candidates  = [[(id,logp,beam_position) for id,logp,beam_position in zip(topk_id.squeeze(),topk_logp.squeeze(),range(beam_size))]]
        beam_batch = [torch.tensor([c[0] for c in candidates[-1]]).to(self.device).view(beam_size,1)]
      prev_logp = torch.tensor([c[1] for c in candidates[-1]]).view(-1,1).to(self.device)
      for t in range(max_predicted_char):
        out_char_embeddings = self.char_embeddings(beam_batch[-1])
        if beam_size == 1:
          out_char_embeddings = out_char_embeddings.view(1,1,-1)
        out,(h_n_dc,c_n_dc) = self.decoder_lstm(out_char_embeddings,(h_n_dc,c_n_dc))
        
        logp = self.distribution_nn(out) #beam_size ,seq_len = 1,vocab_size
        logp = logp.squeeze() #beam_size ,vocab_size
        cumulative_logp = 1/(t+1) * (prev_logp + logp) if beam_size >1 else logp #normalized by current seq length,so it won't prefer generating long sentence
        new_candidate = self.find_topk(cumulative_logp = cumulative_logp,beam_size = beam_size)
        new_beam_batch = [c[0] for c in new_candidate if c[0] != self.EOP_id] #only candidate with no EOP mark will be added to the new_beam_batch
        if new_beam_batch == []: #if new_beam_batch is empty,that means all possible candidates beam reach the EOP mark
          break
        new_beam_batch = torch.tensor(new_beam_batch).view(len(new_beam_batch),1).to(self.device)
        prev_logp = torch.tensor([c[1] for c in new_candidate if c[0] != self.EOP_id]).view(-1,1).to(self.device)
        beampos = torch.tensor([c[2] for c in new_candidate if c[0]!= self.EOP_id]).to(self.device) #the beam position of ecah candidate indicates its "parent node"
        #modifier the hidden and cell state according the beampos of the current candidate
        #this step is crucial for getting the right result 
        h_n_dc = torch.index_select(input = h_n_dc,dim = 1,index = beampos) if beam_size > 1 else h_n_dc
        c_n_dc = torch.index_select(input = c_n_dc,dim = 1,index = beampos) if beam_size > 1 else c_n_dc
        candidates.append(new_candidate)
        beam_batch.append(new_beam_batch)
      
      #this use to retrieve the all possible generated text,including those candidate who reach EOP before the max generation length
      #it will return a list of tuple  : (log_proba,text_ids)
      prediction = self.get_all_predictions(candidates,self.EOP_id) 
      #get the best text_ids
      best_guess = max(prediction,key=lambda x : x[0])
      #convert text_ids to text
      prediction = (best_guess[0],ids_to_texts([[int(i) for i in best_guess[1]]]))

    return prediction
        
      
      

In [ ]:
model = Model(vocabulary_size=len(char_vocabulary), EOP_id=EOP_id, embedding_size=19, lstm_hidden_size=13, lstm_layers=7 ,encoder_dropout_p = 0.2,decoder_dropout_p = 0.1,device='cpu')

In [ ]:

in_paragraphs = torch.tensor([(list(range(5)) + ([batch_generator.padding_idx] * 0))]).to(model.device)
print(in_paragraphs)
out_paragraphs = in_paragraphs
model(in_paragraphs, out_paragraphs)
in_paragraphs = torch.tensor([(list(range(5)) + ([batch_generator.padding_idx] * 10)), (list(range(10)) + ([batch_generator.padding_idx] * 5))]).to(model.device)
print(in_paragraphs)
out_paragraphs = in_paragraphs
model(in_paragraphs, out_paragraphs)

tensor([[0, 1, 2, 3, 4]])
tensor([[  0,   1,   2,   3,   4, 102, 102, 102, 102, 102, 102, 102, 102, 102,
         102],
        [  0,   1,   2,   3,   4,   5,   6,   7,   8,   9, 102, 102, 102, 102,
         102]])


tensor([[[-4.8225, -4.4833, -4.9103,  ..., -4.8653, -4.4773, -4.4805],
         [-4.8453, -4.5073, -4.9151,  ..., -4.8518, -4.5073, -4.4790],
         [-4.8557, -4.5253, -4.9098,  ..., -4.8471, -4.5250, -4.4802],
         ...,
         [-4.8976, -4.4639, -4.9351,  ..., -4.8051, -4.5296, -4.4583],
         [-4.8976, -4.4639, -4.9351,  ..., -4.8051, -4.5296, -4.4583],
         [-4.8976, -4.4639, -4.9351,  ..., -4.8051, -4.5296, -4.4583]],

        [[-4.8316, -4.4788, -4.9124,  ..., -4.8646, -4.4813, -4.4732],
         [-4.8502, -4.5034, -4.9191,  ..., -4.8496, -4.5077, -4.4752],
         [-4.8559, -4.5177, -4.9235,  ..., -4.8433, -4.5194, -4.4759],
         ...,
         [-4.8725, -4.5295, -4.9197,  ..., -4.8332, -4.5362, -4.4766],
         [-4.8723, -4.5272, -4.9185,  ..., -4.8377, -4.5373, -4.4743],
         [-4.8727, -4.5261, -4.9178,  ..., -4.8390, -4.5382, -4.4727]]],
       grad_fn=<LogSoftmaxBackward0>)

In [ ]:
#The vectorized beam search do not support batch generation,the pro is ,since it dose not use loop,so the generation is much quicker
#lease keep batch size = 1 for generating text
batch = batch_generator.get_batch(2)
for b  in batch:
  t  =  model(b[0].view(1,-1).to(model.device), max_predicted_char=16)
  print(t)



(tensor([[-4.3680]], grad_fn=<TopkBackward0>), ['UUUÔÔÔÔÔÔÔÔÔÔÔÔÔx'])
(tensor([[-4.3665]], grad_fn=<TopkBackward0>), ['UUUÔÔÔÔÔÔÔÔÔÔÔÔÔÔ'])


Training
==

In [ ]:
# 'logits' is of shape (batch size, max paragraph length, vocabulary size)
# 'out_paragraphs' is of shape (batch size, max paragraph length)

def loss_function_logits(logits, out_paragraphs,pad_id = PAD_id):
  # Use 'torch.nn.functional.nll_loss'.
  # Computes an average over all tokens of the batch, but do not take into account distribution logits corresponding to padding characters.
  #################
  if logits.size(1) < out_paragraphs.size(1):
    out_paragraphs = out_paragraphs[:,:-1]
  batch_size,max_length,vocabulary_size = logits.size()
  logits = logits.reshape(batch_size*max_length,vocabulary_size)
  golden = out_paragraphs.reshape(batch_size*max_length)
  #pad_mask = golden != pad_id
  return torch.nn.functional.nll_loss(logits,golden,ignore_index=pad_id)
  #################



In [ ]:
def evaluation(prediction,gold):
      p_max  = torch.argmax(prediction,dim  = 2)
      nb_correct = 0
      total = 0
      current_batch_size,seq_len  = p_max.size()
      flatten_pred  = p_max.view(current_batch_size*seq_len)
      flatten_out =  gold[:,:-1].reshape(current_batch_size*seq_len) if  seq_len< gold.size(1) else  gold.reshape(batch_size*seq_len)
      mask  = (flatten_out!=PAD_id)
      nb_correct +=torch.sum((flatten_out==flatten_pred)*mask)
      total   += torch.sum(mask)
      accurracy = float(nb_correct)/float(total)
      return accurracy



In [ ]:
batch = batch_generator.get_batch(2)
logits = model(batch[0].to(model.device), batch[1][:,:-1].to(model.device))
loss_function_logits(logits, batch[1].to(model.device))

tensor(4.7157, grad_fn=<NllLossBackward0>)

In [ ]:
model = Model(vocabulary_size=len(char_vocabulary), EOP_id=EOP_id, embedding_size=256, lstm_hidden_size=512, lstm_layers=1 ,encoder_dropout_p = 0.2,decoder_dropout_p = 0.1,device='cuda')

import time

ratio =  lambda x : int(len(x)*0.9)

batch_generators = []
data = [paragraphs,paragraphs_2]
for  d in data:
    divide  = ratio(d)
    batch_generators.append((BatchGenerator(paragraphs=d[:divide], char_vocabulary=char_vocabulary),
                             BatchGenerator(paragraphs=d[divide:], char_vocabulary=char_vocabulary)))
    
for b in batch_generators:
    print(f"BOOK:{b[0].length()}")
            


model.eval() 

# Training procedure
learning_rate = 0.2
momentum = 0.99 # 0.99
l2_reg = 0.0001
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum, weight_decay=l2_reg) # Once the backward propagation has been done, call the 'step' method (with no argument) to update the parameters.
#optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate,weight_decay=l2_reg)
batch_size = 64
subset = None # Use an integer to train on a smaller portion of the training set, otherwise use None.

for batch_generator in batch_generators:
    epoch_size = batch_generator[0].length() if(subset is None) else subset # In number of instances
    nb_epoch = 20
    epoch_id = 0 # Id of the current epoch
    instances_processed = 0 # Number of instances trained on in the current epoch
    epoch_loss = [] # Will contain the loss for each batch of the current epoch
    time_0 = time.time()

    while(epoch_id < nb_epoch):

      model.train()
      
      model.zero_grad() 

      batch = batch_generator[0].get_batch(batch_size, subset=subset)
      #print(ids_to_texts(batch[0])); print(ids_to_texts(batch[1]))
      in_paragraphs = batch[0].to(model.device)
      out_paragraphs = batch[1].to(model.device)
      

      ###################
      prediction = model(in_paragraphs,out_paragraphs)
      loss = loss_function_logits(prediction,out_paragraphs).to(model.device)
      epoch_loss.append(loss.item())
      loss.backward()
      ###################
      
      optimizer.step() # Updates the parameters.
              
      instances_processed += batch_size
      if(instances_processed > epoch_size):
        print(f"-- END OF EPOCH {epoch_id}.")
        print(f"Average loss on training set: {sum(epoch_loss) / len(epoch_loss)}.")

        model.eval()
        validationn_batch = batch_generator[1].all_batches(32, subset=subset)
        val_loss = []
        val_accuracy = []
        for b in validationn_batch:
          val_in = b[0].to(model.device)
          val_out = b[1].to(model.device)
          val_prediction = model(val_in,val_out)
          val_loss.append( loss_function_logits(val_prediction,val_out).to(model.device).item())
          val_accuracy.append(evaluation(val_prediction,val_out))
        
        
        print(f"Average accuracy on validation set:{sum(val_accuracy)/len(val_accuracy)}")
        print(f"Average loss on validation set: {sum(val_loss) / len(val_loss)}.")
        duration = time.time() - time_0
        print(f"{duration} s elapsed (i.e. {duration / (epoch_id + 1)} s/epoch)")


        # Example of generation
        batch = batch_generator[0].get_batch(1, subset=subset)
        first = ids_to_texts(batch[0])
        second = ids_to_texts(batch[1])
        generated = model(batch[0].to(model.device), max_predicted_char=512)
        score = sentence_bleu(second, generated[1])
        print(f"First: {first}") # Input paragraph
        print(f"Second: {second}")
        print(f"Generated: {generated[1]}") # Generated output paragraph.
        print(f"BLEU score: {score}")

        epoch_id += 1
        instances_processed -= epoch_size
        epoch_loss = []
        epoch_accuracy  = []

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


BOOK:2694
BOOK:2589
-- END OF EPOCH 0.
Average loss on training set: 3.012395187865856.
Average accuracy on validation set:0.29237121699484725
Average loss on validation set: 2.460491895675659.
63.578672647476196 s elapsed (i.e. 63.578672647476196 s/epoch)
First: ['Il lut:\n']
Second: ["-- «Malgré les préjugés qui recouvrent encore une partie de la face de l'Europe comme un réseau, la lumière cependant commence à pénétrer dans nos campagnes. C'est ainsi que, mardi, notre petite cité d'Yonville s'est vue le théâtre d'une expérience chirurgicale qui est en même temps un acte de haute philanthropie. M. Bovary, un de nos praticiens les plus distingués...»\n"]
Generated: ['ZZZnt part part part part part part part part part part part part part part part part part part part part part part part part part part part part part part part part part part part part part part part part part part part part part part part part part part part part part part part part part part part part part part part pa

In [ ]:
#The vectorized beam search do not support batch generation,the pro is ,since it dose not use loop,so the generation is much quicker
test_generator = BatchGenerator(paragraphs = paragraphs, char_vocabulary = char_vocabulary)
text = ["Charles Bovary sortit une bonne bouteille de vin et alla chercher des verres pour ses invités."]
print(text)
for i in range(10):
  batch = test_generator.turn_into_batch(text)
  text = model(batch.to(model.device),beam_size  = 30, max_predicted_char=1024)[1]
  print(text)



['Charles Bovary sortit une bonne bouteille de vin et alla chercher des verres pour ses invités.']
["é le mourant de l'ét l'autre"]
["e l'évêque entra dans la chambre de loite de l'étranger avant d'autour de l'évêque"]
['utre']
["e l'évêque era dans l'ambre de l'étroite de l'évêque en'étévêque entra des pro pays, et leétatroite de l'étroite de l'évêque entra dans l'ouverture de l'évêque entra dans l'ombre de l'étroite de l'étr son coeur de l'auberge.e l'étroite de la "]
["de l'évêque enroite de l'étroite de l'évêque entra porte de l'évêque enra dans la chambre de l'étroite de l'étroite de l'évêque et lui demanda la de l'ét l'ae l'étroite de l'autre"]
["e l'évêque entra dans l'ombre de l'étroite de l'étranger au moment oe l'étrotroite de l'évêque, "]
["ns l'ouverture de l'évêque et l de sa porte de l'étranger avanévêque entre ltroite dbre de l'évêque ene l'auberge de l'évêque entra dans l'ombre de l'évêque et "]
["e l'évêque evée,'ae main slite de l'évêque entra dans le place de l'étroi